## Importing the Data

In [1]:
!pip install transformers

     |████████████████████████████████| 2.1MB 18.5MB/s 
     |████████████████████████████████| 901kB 49.1MB/s 
     |████████████████████████████████| 3.3MB 50.6MB/s 


In [2]:
import os
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
from torch.utils.data import (TensorDataset, DataLoader, SequentialSampler)
from transformers import BertModel, BertTokenizer

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Constants
DATASET_DIR = '/content/drive/MyDrive/ColabNotebooks/AES-for-Korean/data'
SAVE_DIR = '/content/drive/MyDrive/ColabNotebooks/AES-for-Korean/data'

X = pd.read_csv(os.path.join(DATASET_DIR, 'training_set_rel.tsv'), sep='\t', encoding='ISO-8859-1')
X = X.dropna(axis=1)
X = X.drop(columns=['rater1_domain1', 'rater2_domain1'])

In [ ]:
X.head()

,essay_id,essay_set,essay,domain1_score
0,1,1,"Dear local newspaper, I think effects computer...",8
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10
4,5,1,"Dear @LOCATION1, I know having computers has a...",8


Minimum and Maximum Scores for each essay set.

In [ ]:
minimum_scores = np.array([-1, 2, 1, 0, 0, 0, 0, 0, 0])
maximum_scores = np.array([-1, 12, 6, 3, 3, 4, 4, 30, 60])

In [ ]:
old_min = minimum_scores[X['essay_set']]
old_max = maximum_scores[X['essay_set']]
old_range = old_max - old_min
new_min = 0
new_max = 1
new_range = (new_max - new_min)  
X['score'] = (((X['domain1_score'] - old_min) * new_range) / old_range) + new_min

# round score to nearest integer for cohen kappa calculation
y = X['score']

X.head()

,essay_id,essay_set,essay,domain1_score,score
0,1,1,"Dear local newspaper, I think effects computer...",8,0.6
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,0.7
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,0.5
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10,0.8
4,5,1,"Dear @LOCATION1, I know having computers has a...",8,0.6


## Preprocessing the Data

We will preprocess all essays and convert them to feature vectors so that they can be fed into the RNN.

These are all helper functions used to clean the essays.

In [ ]:
import nltk

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import numpy as np
import re
from nltk.corpus import stopwords

def essay_to_wordlist(essay_v, remove_stopwords):
    """Remove the tagged labels and word tokenize the sentence."""
    essay_v = re.sub("[^a-zA-Z]", " ", essay_v)
    words = essay_v.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return (words)

def essay_to_sentences(essay_v, remove_stopwords):
    """Sentence tokenize the essay and call essay_to_wordlist() for word tokenization."""
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(essay_v.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            tokenized_sentences = essay_to_wordlist(raw_sentence, remove_stopwords)
            if tokenized_sentences:
                sentences.append(tokenized_sentences)
    return sentences

def makeFeatureVec(words, model, num_features):
    """Make Feature Vector from the words list of an Essay."""
    featureVec = np.zeros((num_features,),dtype="float32")
    num_words = 0.
    for word in words:
        if word in model:
            num_words += 1
            featureVec = np.add(featureVec, model[word])       
    featureVec = np.divide(featureVec,num_words)
    return featureVec

def makeFeatureVec2(words, model, num_features):
    """Make Feature Vector from the words list of an Essay."""
    featureVec = np.zeros((num_features,),dtype="float32")
    for word in words:
        if word in model:
            featureVec = np.add(featureVec, model[word])
    if len(words) != 0:
        featureVec = np.divide(featureVec,float(len(words)))
    return featureVec

def makeFeatureVec3(words, model, num_features):
    """Make Feature Vector from the words list of an Essay."""
    featureVec = []
    for word in words:
        if word in model:
            featureVec.append(np.array(model[word], dtype="float32"))
    return featureVec

def getAvgFeatureVecs(essays, model, num_features):
    """Main function to generate the word vectors for glove model."""
    counter = 0
    essayFeatureVecs = np.zeros((len(essays),num_features),dtype="float32")
    for essay in essays:
        essayFeatureVecs[counter] = makeFeatureVec(essay, model, num_features)
        counter = counter + 1
    return essayFeatureVecs

def getAvgFeatureVecs2(essay, model, num_features):
    """Main function to generate the word vectors for glove model."""
    essayFeatureVecs = np.zeros((len(essay),num_features),dtype="float32")
    for cnt, sentence in enumerate(essay):
        essayFeatureVecs[cnt] = makeFeatureVec2(sentence, model, num_features)
    return essayFeatureVecs

In [ ]:
pretrained_model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(pretrained_model_name)
model = BertModel.from_pretrained(pretrained_model_name).cuda()
#model.train()

model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [ ]:
from sklearn.model_selection import KFold
from keras.preprocessing.sequence import pad_sequences

essay_data = X['essay']
is_remove_stopwords = False
tokenized_essay = []
padded_tokenized_essay = []
attention_mask_essay = []
sent_max_len = 200
for ix, essay in enumerate(essay_data):
  if ix % 1000 == 0:
    print(ix)
  sentences = essay_to_sentences(essay, remove_stopwords=is_remove_stopwords)
  tokenized_sentences = []
  for iy, sentence in enumerate(sentences):
      tokenized_sentence = np.array(tokenizer.encode(sentence, add_special_tokens=True))
      tokenized_sentences.append(tokenized_sentence)
  padded_tokenized_sentences = pad_sequences(tokenized_sentences, maxlen=sent_max_len, padding='post')
  attention_mask_sentences = np.where(padded_tokenized_sentences != 0, 1, 0)
  #padded_tokenized_sentences = torch.tensor(pad_sequences(tokenized_sentences, maxlen=sent_max_len, padding='post'))
  #attention_mask_sentences = torch.tensor(np.where(padded_tokenized_sentences != 0, 1, 0))
  padded_tokenized_essay.append(padded_tokenized_sentences)
  attention_mask_essay.append(attention_mask_sentences)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000


In [ ]:
tttt_essays = []
tttt = 0
max_tttt = 0
for essay in essay_data:
  tttt_tmp = essay_to_sentences(essay, remove_stopwords=False)
  tttt_essays.append(tttt_tmp)
  tttt += len(tttt_tmp)
  if len(tttt_tmp) > max_tttt:
    max_tttt = len(tttt_tmp)

In [ ]:
padded_tokenized_essay[0].shape

(16, 200)

In [ ]:
print(max_tttt)
print(X['essay'][0])
print(tttt_essays[0])

96
Dear local newspaper, I think effects computers have on people are great learning skills/affects because they give us time to chat with friends/new people, helps us learn about the globe(astronomy) and keeps us out of troble! Thing about! Dont you think so? How would you feel if your teenager is always on the phone with friends! Do you ever time to chat with your friends or buisness partner about things. Well now - there's a new way to chat the computer, theirs plenty of sites on the internet to do so: @ORGANIZATION1, @ORGANIZATION2, @CAPS1, facebook, myspace ect. Just think now while your setting up meeting with your boss on the computer, your teenager is having fun on the phone not rushing to get off cause you want to use it. How did you learn about other countrys/states outside of yours? Well I have by computer/internet, it's a new way to learn about what going on in our time! You might think your child spends a lot of time on the computer, but ask them so question about the econ

# BERT Embedding

You need to execute this section only once.

In [ ]:
import csv
ff = open(os.path.join(SAVE_DIR, 'embedded_features_special_tokens.csv'), 'a', newline='')
writer_ff = csv.writer(ff)

In [ ]:
import psutil

with torch.no_grad():
  print("ix, gpu memory, cpu usage, vm usage, data size ")
  for ix in range(len(padded_tokenized_essay)):
    if ix % 100 == 0:
      print(ix, torch.cuda.memory_allocated(0), psutil.cpu_percent(), psutil.virtual_memory()[2])
    sen = torch.tensor(padded_tokenized_essay[ix]).cuda()
    mask = torch.tensor(attention_mask_essay[ix]).cuda()
    last_hidden_states_train = model(sen, attention_mask=mask)
    embedded_features = last_hidden_states_train[0].detach().cpu()[:, 0, :].numpy()
    # embedded_essay.append(embedded_features)
    for i in embedded_features:
      writer_ff.writerow(i)
    #np.savetxt(os.path.join(SAVE_DIR, 'embedded_features.csv'), embedded_features, delimiter=",")
    del sen, mask, last_hidden_states_train, embedded_features
    torch.cuda.empty_cache()

torch.cuda.empty_cache()

ix, gpu memory, cpu usage, vm usage, data size 
0 439065600 31.3 17.0
100 439065600 53.2 16.9
200 439065600 53.1 16.9
300 439065600 53.2 16.9
400 439065600 53.2 16.9
500 439065600 53.3 16.9
600 439065600 53.1 16.9
700 439065600 53.2 16.9
800 439065600 53.3 16.9
900 439065600 53.1 16.9
1000 439065600 53.2 16.9
1100 439065600 53.2 16.9
1200 439065600 53.1 16.9
1300 439065600 53.2 16.9
1400 439065600 53.2 16.9
1500 439065600 53.2 16.9
1600 439065600 53.3 16.9
1700 439065600 53.2 16.9
1800 439065600 53.2 16.9
1900 439065600 53.2 16.9
2000 439065600 53.3 16.9
2100 439065600 53.3 16.9
2200 439065600 53.2 16.9
2300 439065600 53.3 16.9
2400 439065600 53.3 16.9
2500 439065600 53.4 16.9
2600 439065600 53.3 16.9
2700 439065600 53.4 16.9
2800 439065600 53.4 16.9
2900 439065600 53.4 16.9
3000 439065600 53.3 16.9
3100 439065600 53.1 16.9
3200 439065600 53.3 16.9
3300 439065600 53.2 16.9
3400 439065600 53.3 16.9
3500 439065600 53.2 16.9
3600 439065600 53.4 16.9
3700 439065600 53.4 16.9
3800 439065600

In [ ]:
ff.close()

# Load the embedded data

In [ ]:
embedded_essay_raw = pd.read_csv(os.path.join(DATASET_DIR, 'embedded_features_special_tokens.csv'), sep=',', encoding='ISO-8859-1')
#embedded_essay_raw = pd.read_csv(os.path.join(DATASET_DIR, 'embedded_features_raw.csv'), sep=',', encoding='ISO-8859-1')
#embedded_essay_raw = pd.read_csv(os.path.join(DATASET_DIR, 'embedded_features.csv'), sep=',', encoding='ISO-8859-1')
embedded_essay = []

In [ ]:
embedded_essay_raw.shape

(164770, 768)

In [ ]:
from keras.preprocessing.sequence import pad_sequences

tmp_ix = 0
for ix, essay_raw in enumerate(padded_tokenized_essay):
  if ix % 500 == 0:
    print(ix)
  tmp_len = len(essay_raw)
  essay = embedded_essay_raw[tmp_ix:tmp_ix + tmp_len]
  embedded_essay.append(essay)
  tmp_ix += tmp_len

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500


In [ ]:
#embedded_essay = pad_sequences(embedded_essay, maxlen=96, padding='pre', dtype='float')

Now we train the model on the dataset.

We will use 5-Fold Cross Validation and measure the Quadratic Weighted Kappa for each fold.
We will then calculate Average Kappa for all the folds.

## Defining the model 

Here we define a 2-Layer LSTM Model. 

Note that instead of using sigmoid activation in the output layer we will use
Relu since we are not normalising training labels.

In [ ]:
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense, Dropout, Lambda, Flatten
from tensorflow.keras.models import Sequential, load_model, model_from_config
import tensorflow.keras.backend as K

def get_model():
    """Define the model."""
    model = Sequential()
    model.add(LSTM(200, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 200], return_sequences=True))
    model.add(LSTM(64, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))

    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])
    model.summary()

    return model

def get_sentence_model():
    """Define the model."""
    model = Sequential()
    model.add(GRU(128, dropout=0.4, input_shape=[128, 768], return_sequences=True))
    model.add(GRU(64))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))

    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])
    model.summary()

    return model

def get_sentence_model2():
    """Define the model."""
    model = Sequential()
    model.add(GRU(96, dropout=0.4, input_shape=[128, 768], return_sequences=True))
    model.add(GRU(64))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))

    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])
    model.summary()

    return model

def get_sentence_model3():
    """Define the model."""
    model = Sequential()
    model.add(GRU(256, dropout=0.4, input_shape=[128, 768], return_sequences=True))
    model.add(GRU(128, dropout=0.4, return_sequences=True))
    model.add(GRU(64))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))

    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])
    model.summary()

    return model

## Fine-tuning BERT + GRU 2 Layer with PyTorch

In [ ]:
import torch.nn as nn
from transformers import AutoConfig, AutoModel

class 


## Freezed Bert + GRU 2 Layer with Keras

In [ ]:
#embedded_essay = pad_sequences(embedded_essay, maxlen=96, padding='pre', dtype='float')

import keras
import math

class DataGenerator(keras.utils.Sequence):

  def __init__(self, ids, batch_size=64, shuffle=True):
    self.ids = ids
    self.batch_size = batch_size
    self.shuffle = shuffle
    self.on_epoch_end()

  def on_epoch_end(self):
    # Updates indexes after each epoch
    self.indexes = np.arange(len(self.ids))
    if self.shuffle == True:
      np.random.shuffle(self.indexes)

  def __len__(self):
    return math.ceil(len(self.ids) / self.batch_size)

  def __getitem__(self, index):
    # Generated data containing batch_size samples
    batch_ids = self.ids[index * self.batch_size:(index + 1) * self.batch_size]

    essays = list()
    scores = list()
    for ix in batch_ids:
      essay = embedded_essay[ix]
      score = y[ix]
      essays.append(essay)
      scores.append(score)
    essays = pad_sequences(essays, maxlen=128, padding='pre', dtype='float')

    return np.array(essays), np.array(scores)

In [ ]:
y
len(embedded_essay)

12976

In [ ]:
embedded_essay[0]

,0.03897584,0.1705755,0.16306049,-0.32822582,-0.4931742,-0.5777212,0.51011056,0.8981184,0.15726711,-0.63853836,-0.16852458,-0.2539917,0.0052978797,0.3112666,0.30889437,-0.23946148,-0.5916331,0.5375616,0.10453381,0.25923082,0.040120613,-0.35811937,0.084900536,-0.12979448,0.20781113,-0.24564661,-0.21821417,-0.18564206,-0.10883415,0.19866161,-0.2736097,0.4945069,-0.06111347,0.09564236,0.044967655,-0.49912897,0.44235414,0.23340337,0.18972217,0.2369034,...,0.17444798,-0.7208587,0.2390435,0.5639985,-0.02137434,-0.05141661,-0.030394815,-0.104016826,-0.5468714,-0.16815141,-0.19427754,0.38222885,0.12647282,0.095467836,-0.37149695,0.43112832,0.28849825,0.55694956,0.24611443,-0.053098254,-0.10154303,-0.17519955,0.4287814,0.5200037,-5.403975,0.04553558,-0.12436247,0.0050980854,-0.22526489,-0.3898146,-0.115320064,-0.37885273,0.02988772,-0.12692152,0.37245676,0.12877496,-0.19407533,-0.6573706,0.3568359,0.39976805
0,-0.031000,-0.039242,-0.320096,0.006946,-0.196671,-0.396228,0.115823,0.360166,-0.096670,0.039806,0.451213,-0.267765,0.124315,0.249646,-0.367936,-0.043657,-0.022469,0.261491,0.099743,-0.213620,-0.189256,-0.105234,-0.299287,0.109172,-0.152034,-0.175036,0.098903,-0.160676,0.029059,0.270451,0.076392,-0.046378,-0.277001,0.104086,-0.090018,-0.100922,0.131680,0.024087,-0.158221,0.047898,...,-0.010769,0.147479,-0.074286,0.275063,-0.389159,0.159046,-0.019297,-0.175742,0.086575,-0.112550,-0.193133,0.107199,-0.088026,0.029302,0.194172,0.284729,0.118845,0.067868,0.018238,0.029295,-0.110705,-0.286266,-0.202426,-0.104493,-9.110782,-0.043799,-0.111329,0.024805,0.037790,0.062525,0.081043,-0.333182,0.135394,0.122937,-0.028214,-0.039936,-0.211770,-0.091390,0.235144,0.060140
1,0.270068,0.286835,-0.312244,-0.083724,-0.371326,-0.175925,0.310784,0.115957,-0.070045,-0.255145,-0.010835,0.018113,-0.028804,0.281229,0.261835,-0.047650,-0.139911,0.303671,0.401240,0.039095,0.122667,-0.133192,-0.073310,-0.188598,-0.249804,-0.148067,0.042947,-0.225651,0.076616,-0.057281,0.214792,0.259625,-0.402984,0.111978,0.161541,0.090459,0.094468,0.162238,0.264531,0.189335,...,-0.247232,-0.155336,-0.072469,0.505717,-0.052829,0.117284,-0.146075,-0.117058,-0.195858,0.039858,-0.382877,0.371591,0.139386,-0.021546,0.052934,0.125495,0.533277,0.321418,0.190057,-0.311492,-0.189408,-0.094566,0.422168,0.430083,-7.357372,0.060102,-0.136205,-0.376616,-0.042260,-0.169703,0.170489,-0.201935,0.044900,-0.207599,0.126979,0.269769,-0.073095,-0.206223,0.433744,0.234084
2,0.044999,0.103974,-0.273873,0.062710,-0.203671,-0.184251,0.252243,0.580784,-0.227311,-0.122951,0.451432,0.020150,0.049043,0.352111,0.256963,0.119588,-0.190546,0.499676,0.427755,0.083106,-0.256669,-0.257225,-0.117288,0.003841,0.305160,-0.194973,-0.085303,-0.172681,0.266695,-0.238845,-0.007221,0.306875,0.022228,-0.027960,0.263643,-0.086819,0.064743,-0.166808,0.038233,0.162414,...,0.323896,-0.070453,-0.169819,0.171146,0.061148,0.177073,-0.132677,-0.064897,-0.192132,-0.256930,-0.395001,0.397926,0.119960,0.015796,-0.011583,0.267547,0.597411,0.016349,0.398271,0.018641,-0.199599,-0.225537,0.404656,0.197267,-8.405067,0.052658,-0.291597,-0.414220,-0.030366,-0.260031,0.149828,-0.356936,-0.014905,-0.132931,0.180756,0.205888,-0.224388,-0.503448,0.048857,0.244911
3,0.017118,0.121252,-0.644177,-0.120316,-0.136243,-0.494843,0.447419,0.686055,-0.444794,-0.281085,0.090090,-0.233360,-0.042814,0.035806,0.235632,-0.089441,-0.000720,0.561433,0.362729,0.034237,-0.137094,-0.155068,-0.099510,-0.216670,-0.217682,-0.000062,-0.171357,-0.095839,0.192478,-0.204740,-0.212839,0.535659,-0.343187,0.064332,0.146969,0.006006,0.191289,0.089183,0.103925,0.000242,...,0.219435,-0.385291,0.141750,0.083287,-0.065311,0.208528,-0.201536,-0.064400,-0.075216,0.059751,-0.390485,0.333034,0.330601,0.071649,-0.061794,0.281069,0.285943,0.143600,0.656836,-0.196885,-0.124114,-0.092115,0.813195,0.426665,-7.306691,-0.044458,-0.576241,-0.317755,-0.150216,-0.361833,0.193215,-0.388127,0.219215,-0.101050,0.335746,0.036126,-0.130592,-0.519933,0.330751,0.133035
4,0.120396,-0.007655,0.1

In [ ]:
from keras.callbacks import EarlyStopping
from sklearn.metrics import cohen_kappa_score

batch_size = 64

cv = KFold(n_splits=5, shuffle=True)
cnt = 0
for traincv, testcv in cv.split(embedded_essay):
  train_gen = DataGenerator(traincv, batch_size=batch_size)
  test_gen = DataGenerator(testcv, batch_size=batch_size, shuffle=False)
  train_y = y.iloc[traincv]
  test_y = y.iloc[testcv]

  train_steps = len(traincv) // batch_size
  valid_steps = len(testcv) // batch_size

  early_stopping = EarlyStopping(monitor='loss', min_delta=0, patience=5, verbose=0, mode='auto')
  sentence_model = get_sentence_model3()
  sentence_model.fit(train_gen, steps_per_epoch=train_steps, validation_steps=valid_steps,
                    epochs=50, callbacks=[early_stopping])

  y_sent_pred = sentence_model.predict(test_gen) *100
  y_sent_pred = np.round(y_sent_pred)

  sentence_result = cohen_kappa_score(np.array(np.round(test_y * 100)), y_sent_pred, weights='quadratic')
  print("Kappa Score", cnt, ": {}".format(sentence_result))
  cnt += 1

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (None, 128, 256)          787968    
_________________________________________________________________
gru_1 (GRU)                  (None, 128, 128)          148224    
_________________________________________________________________
gru_2 (GRU)                  (None, 64)                37248     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 973,505
Trainable params: 973,505
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
162/162 [==============================] - 49s 98ms/step - loss: 0.3217 - mae: 0.4667
Epoch 2/50
162/162 

In [ ]:
from keras.callbacks import EarlyStopping
from sklearn.metrics import cohen_kappa_score

batch_size = 64

cv = KFold(n_splits=5, shuffle=True)
cnt = 0
for traincv, testcv in cv.split(embedded_essay):
  train_gen = DataGenerator(traincv, batch_size=batch_size)
  test_gen = DataGenerator(testcv, batch_size=batch_size, shuffle=False)
  train_y = y.iloc[traincv]
  test_y = y.iloc[testcv]

  train_steps = len(traincv) // batch_size
  valid_steps = len(testcv) // batch_size

  early_stopping = EarlyStopping(monitor='loss', min_delta=0, patience=5, verbose=0, mode='auto')
  sentence_model = get_sentence_model3()
  sentence_model.fit(train_gen, steps_per_epoch=train_steps, validation_steps=valid_steps,
                    epochs=50, callbacks=[early_stopping])

  y_sent_pred = sentence_model.predict(test_gen) *100
  y_sent_pred = np.round(y_sent_pred)

  sentence_result = cohen_kappa_score(np.array(np.round(test_y * 100)), y_sent_pred, weights='quadratic')
  print("Kappa Score", cnt, ": {}".format(sentence_result))
  cnt += 1

Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_52 (GRU)                 (None, 128, 256)          787968    
_________________________________________________________________
gru_53 (GRU)                 (None, 128, 128)          148224    
_________________________________________________________________
gru_54 (GRU)                 (None, 64)                37248     
_________________________________________________________________
dropout_26 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_26 (Dense)             (None, 1)                 65        
Total params: 973,505
Trainable params: 973,505
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
162/162 [==============================] - 11s 48ms/step - loss: 0.3405 - mae: 0.5058
Epoch 2/50
162/1

In [ ]:
from keras.callbacks import EarlyStopping
from sklearn.metrics import cohen_kappa_score

batch_size = 64

cv = KFold(n_splits=5, shuffle=True)
cnt = 0
for traincv, testcv in cv.split(embedded_essay):
  train_gen = DataGenerator(traincv, batch_size=batch_size)
  test_gen = DataGenerator(testcv, batch_size=batch_size, shuffle=False)
  train_y = y.iloc[traincv]
  test_y = y.iloc[testcv]

  train_steps = len(traincv) // batch_size
  valid_steps = len(testcv) // batch_size

  early_stopping = EarlyStopping(monitor='loss', min_delta=0, patience=5, verbose=0, mode='auto')
  sentence_model = get_sentence_model()
  sentence_model.fit(train_gen, steps_per_epoch=train_steps, validation_steps=valid_steps,
                    epochs=50, callbacks=[early_stopping])

  y_sent_pred = sentence_model.predict(test_gen) *100
  y_sent_pred = np.round(y_sent_pred)

  sentence_result = cohen_kappa_score(np.array(np.round(test_y * 100)), y_sent_pred, weights='quadratic')
  print("Kappa Score", cnt, ": {}".format(sentence_result))
  cnt += 1

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_20 (GRU)                 (None, 96, 128)           344832    
_________________________________________________________________
gru_21 (GRU)                 (None, 64)                37248     
_________________________________________________________________
dropout_10 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 65        
Total params: 382,145
Trainable params: 382,145
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
162/162 [==============================] - 9s 37ms/step - loss: 0.3627 - mae: 0.4369
Epoch 2/50
162/162 [==============================] - 6s 37ms/step - loss: 0.0645 - mae: 0.2000
Epoch 3/50
162/162 [==============================] -

In [ ]:
test_y = y.iloc[testcv]
sentence_result = cohen_kappa_score(np.array(np.round(test_y * 100)), y_sent_pred, weights='quadratic')
print("Kappa Score 0", ": {}".format(sentence_result))

Kappa Score 0 : 0.7346137426085717


In [ ]:
print(np.array(np.round(test_y * 100))[0:100])
print(y_sent_pred[0:100])

[ 70.  60.  60.  70.  70.  70.  60.  80.  70.  20.  70.  60.  60.  80.
  60.  60.  70.  60.  60. 100.  60.  50.  50.  50.  70.  60.  60. 100.
  80.  60.  70.  80.  70.  80.  60.  60.  80.  60.  30.  70.  80.  60.
  80.  80.  60.  40.  40.  60.  70.  70.  70.  60.  60.  70.  50.  90.
  60.  60.  80.  80.  80.  70.  70.  80.  60.  60.  60.  60.  70.  90.
  60.  50.  40.  80.  60.  40.  60.  40.  80.  60.  60.  80.  70.  70.
  60.  60.  60. 100.  80.  60.  70.  60.  50. 100.  70.  60.  60.  70.
  60.  40.]
[[64.]
 [69.]
 [51.]
 [68.]
 [68.]
 [56.]
 [56.]
 [87.]
 [77.]
 [31.]
 [69.]
 [72.]
 [78.]
 [64.]
 [60.]
 [66.]
 [73.]
 [55.]
 [59.]
 [83.]
 [59.]
 [55.]
 [67.]
 [52.]
 [72.]
 [49.]
 [51.]
 [70.]
 [66.]
 [47.]
 [76.]
 [89.]
 [83.]
 [75.]
 [73.]
 [63.]
 [85.]
 [58.]
 [10.]
 [68.]
 [83.]
 [62.]
 [63.]
 [69.]
 [62.]
 [53.]
 [47.]
 [70.]
 [66.]
 [79.]
 [66.]
 [60.]
 [56.]
 [58.]
 [57.]
 [72.]
 [66.]
 [58.]
 [83.]
 [64.]
 [84.]
 [78.]
 [71.]
 [59.]
 [52.]
 [67.]
 [54.]
 [66.]
 [65.]
 [74.]
 

## Training Phase - Sentence

문장 단위, 모델 돌리는 부분만 (전처리는 위에서)